In [10]:
import sys
import os
import pandas as pd
import requests
from IPython.display import display

# Add project root to sys.path to allow imports from 'src'
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

from src.edge_fetcher import get_skater_edge_data

# Set pandas display options for better notebook visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0'})

# Retrieve NHL Edge data for Connor McDavid (ID: 8478402)
raw_data = get_skater_edge_data(8478402, session)

if raw_data:
    # Use json_normalize to flatten nested keys and transpose for vertical readability
    speed_data = raw_data.get('skatingWheel', raw_data) 
    df_flat = pd.json_normalize(speed_data).T.reset_index()
    
    # Improvement: Organized columns for cleaner metric reporting
    df_flat.columns = ['Metric', 'McDavid Value']
    
    print(f"--- NHL Edge Analysis: {raw_data.get('objectFullName')} ---")
    display(df_flat)
else:
    print("Error: Failed to retrieve data from the NHL API.")

--- NHL Edge Analysis: None ---


,Metric,McDavid Value
0,seasonsWithEdgeStats,"[{'id': 20212022, 'gameTypes': [2, 3]}, {'id': 20222023, 'gameTypes': [2, 3]}, {'id': 20232024, 'gameTypes': [2, 3]}, {'id': 20242025, 'gameTypes': [2, 3]}, {'id': 20252026, 'gameTypes': [2]}]"
1,sogSummary,"[{'locationCode': 'all', 'shots': 151, 'shotsPercentile': 0.9945, 'shotsLeagueAvg': 48.8, 'goals': 25, 'goalsPercentile': 0.9945, 'goalsLeagueAvg': 6.1909, 'shootingPctg': 0.1656, 'shootingPctgPercentile': 0.7614, 'shootingPctgLeagueAvg': 0.1269}, {'locationCode': 'high', 'shots': 59, 'shotsPercentile': 0.9909, 'shotsLeagueAvg': 17.8709, 'goals': 12, 'goalsPercentile': 0.9745, 'goalsLeagueAvg': 3.5109, 'shootingPctg': 0.2034, 'shootingPctgPercentile': 0.5439, 'shootingPctgLeagueAvg': 0.1965}, {'locationCode': 'long', 'shots': 2, 'shotsPercentile': 0.2878, 'shotsLeagueAvg': 4.6418, 'goals': 1, 'goalsPercentile': 0.8889, 'goalsLeagueAvg': 0.1527, 'shootingPctg': 0.5, 'shootingPctgPercentile': 0.9965, 'shootingPctgLeagueAvg': 0.0329}, {'locationCode': 'mid', 'shots': 38, 'shotsPercentile': 0.9235, 'shotsLeagueAvg': 15.3218, 'goals': 4, 'goalsPercentile': 0.8361, 'goalsLeagueAvg': 1.7873, 'shootingPctg': 0.1053, 'shootingPctgPercentile': 0.4316, 'shootingPctgLeagueAvg': 0.1166}]"
2,sogDetails,"[{'area': 'Behind the Net', 'shots': 4, 'shotsPercentile': 0.9836}, {'area': 'Beyond Red Line', 'shots': 1, 'shotsPercentile': 0.5064}, {'area': 'Center Point', 'shots': 1, 'shotsPercentile': 0.388}, {'area': 'Crease', 'shots': 14, 'shotsPercentile': 0.9964}, {'area': 'High Slot', 'shots': 11, 'shotsPercentile': 0.878}, {'area': 'L Circle', 'shots': 17, 'shotsPercentile': 0.9563}, {'area': 'L Corner', 'shots': 1, 'shotsPercentile': 0.9071}, {'area': 'L Net Side', 'shots': 20, 'shotsPercentile': 1.0}, {'area': 'L Point', 'shots': 1, 'shotsPercentile': 0.3752}, {'area': 'Low Slot', 'shots': 45, 'shotsPercentile': 0.9745}, {'area': 'Offensive Neutral Zone', 'shots': 2, 'shotsPercentile': 0.7013}, {'area': 'Outside L', 'shots': 10, 'shotsPercentile': 0.9636}, {'area': 'Outside R', 'shots': 7, 'shotsPercentile': 0.9217}, {'area': 'R Circle', 'shots': 10, 'shotsPercentile': 0.8251}, {'area': 'R Corner', 'shots': 0, 'shotsPercentile': 0.0}, {'area': 'R Net Side', 'shots': 7, 'shotsPercentile': 0.9763}, {'area': 'R Point', 'shots': 0, 'shotsPercentile': 0.0}]"
3,player.id,8478402
4,player.firstName.default,Connor
5,player.lastName.default,McDavid
6,player.birthDate,1997-01-13
7,player.shootsCatches,L
8,player.sweaterNumber,97
9,player.position,C


In [ ]:
import pandas as pd
import json

# --- GLOBAL CONFIGURATION ---
# Change this variable to analyze a different player
# In a real app, you would swap the entire JSON object here
TARGET_DATA = ""  # Your provided data variable

def process_nhl_data(data):
    # 1. EXTRACT FILENAME (e.g., connor_mcdavid_edge_stats.csv)
    fname = data['player']['firstName']['default'].lower()
    lname = data['player']['lastName']['default'].lower()
    filename = f"{fname}_{lname}_edge_stats.csv"

    # 2. TRANSFORM RAW DATA TO CSV
    # pd.json_normalize flattens the nested keys into "player.team.abbrev" etc.
    df = pd.json_normalize(data)
    df.to_csv(filename, index=False)
    
    # 3. GENERATE NEUTRAL SENTENCES (Internal Python logic)
    # We pull the specific metrics for our own "unflavored" reading
    full_name = f"{data['player']['firstName']['default']} {data['player']['lastName']['default']}"
    
    report = [
        f"REPORT FOR: {full_name}",
        f"- Points: {data['player']['points']} in {data['player']['gamesPlayed']} GP.",
        f"- Maximum Skating Speed: {data['skatingSpeed']['speedMax']['imperial']} mph.",
        f"- Speed Bursts Over 20 mph: {data['skatingSpeed']['burstsOver20']['value']}.",
        f"- Offensive Zone Time: {data['zoneTimeDetails']['offensiveZonePctg'] * 100:.2f}%."
    ]
    
    return filename, "\n".join(report)

# --- EXECUTION ---
csv_name, internal_report = process_nhl_data(TARGET_DATA)

print(f"Success! Raw data saved to: {csv_name}")
print("\n--- INTERNAL NEUTRAL REPORT ---")
print(internal_report)